In [ ]:
# Se importan las librerías

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import glob
import csv
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import numpy as np
from sklearn.metrics import classification_report
import json
import spacy
import random
import re
import codecs
import spacy

In [ ]:
#Cargar los expedientes médicos para revisar el formato en que se encuentran
json_decoded=dict()
with open('Expedientes Medicos.json', "r",encoding="utf-8") as f:
    for i, txt in enumerate( f.readlines()[1:]):
        json_decoded[i]=json.loads(txt)

nota_orig = json_decoded

In [ ]:
# Revisamos la Nota e su vista de ella
nota_orig[0]

{'content': '"Nota Médica < paciente> 7000000000486834 06/09/1962 515351 < paciente> Mujer < doctor> HOSPITAL ABC < doctor> Signos Vitales 21/07/2020 06:52: Temperatura: 36.4 / Frecuencia cardiaca - ADL: 82.0 / Frecuencia respiratoria - ADL: 20.0 / SaO2: 93.0 / Otras constantes de hoy:Tensión Arterial Sistólica - ADL: 125.0 / Tensión Arterial Diastólica - ADL: 80.0 / Tensión Arterial Media - ADL: 95.0 / Síntomas Se trata de < paciente> de 57 años de edad, con Obesidad grado I, sin otros antecedentes de importancia para la enfermedad actual. Niega dolor torácico, Negó sintomatología urinaria digestiva. La paciente niega la presencia de disnea. Objetivo Mujer de edad aparente igual la cronológica, orientada en tiempo, persona, lugar circunstancia, alerta. Coloración normal de mucosas. Estado de hidratación adecuado, con aporte de oxígeno suplementario por puntas nasales 0.5lpm . Saturando 96%. Sin datos de dificultad respiratoria. Se encuentra únicamente con aporte de oxigeno suplementar

In [ ]:
# Revisamos como se ve con las palabras unicamente del contenido
nota_med = []
for i in range(len(nota_orig)):nota_med.append(nota_orig[i]['content'].split())
nota_med[0]

['"Nota',
 'Médica',
 '<',
 'paciente>',
 '7000000000486834',
 '06/09/1962',
 '515351',
 '<',
 'paciente>',
 'Mujer',
 '<',
 'doctor>',
 'HOSPITAL',
 'ABC',
 '<',
 'doctor>',
 'Signos',
 'Vitales',
 '21/07/2020',
 '06:52:',
 'Temperatura:',
 '36.4',
 '/',
 'Frecuencia',
 'cardiaca',
 '-',
 'ADL:',
 '82.0',
 '/',
 'Frecuencia',
 'respiratoria',
 '-',
 'ADL:',
 '20.0',
 '/',
 'SaO2:',
 '93.0',
 '/',
 'Otras',
 'constantes',
 'de',
 'hoy:Tensión',
 'Arterial',
 'Sistólica',
 '-',
 'ADL:',
 '125.0',
 '/',
 'Tensión',
 'Arterial',
 'Diastólica',
 '-',
 'ADL:',
 '80.0',
 '/',
 'Tensión',
 'Arterial',
 'Media',
 '-',
 'ADL:',
 '95.0',
 '/',
 'Síntomas',
 'Se',
 'trata',
 'de',
 '<',
 'paciente>',
 'de',
 '57',
 'años',
 'de',
 'edad,',
 'con',
 'Obesidad',
 'grado',
 'I,',
 'sin',
 'otros',
 'antecedentes',
 'de',
 'importancia',
 'para',
 'la',
 'enfermedad',
 'actual.',
 'Niega',
 'dolor',
 'torácico,',
 'Negó',
 'sintomatología',
 'urinaria',
 'digestiva.',
 'La',
 'paciente',
 'niega',
 '

In [ ]:
# Utilizamos un tratamiento de archivos tsv, para corregir daños en los tabuladores,
# en los saltos de línea etc
class TSV(object):
    """
    .tsv file's handler.
    """
 
    def __init__(self, file):
        """
        TSV init.
        :param file: .tsv file to handle.
        """
        self.file = file
 
    def __repr__(self):
        return "File {file} under handling......".format(file=self.file)
 
    def tsv(self):
        """
        .tsv file's column definition and data check.
        :return: List.
            lines data from [file] row by row in dict format.
        """
        with codecs.open(self.file, 'r', "utf-8") as f:
            line = f.readline()
            data = []
            head = []
            while line:
                if line.isspace():
                    line = f.readline()
                    continue
                elif not line.isspace():
                    # to be compatible between OS
                    head = line.rstrip("\r\n").split('\t')
                    line = f.readline()
                    break
            while line:
                if line.isspace():
                    line = f.readline()
                    continue
                elif not line.isspace():
                    body = line.rstrip("\r\n").split('\t')
                    rows = zip(head, body)
                    tsv_dic = {}
                    for (head_sub, body_sub) in list(rows):
                        tsv_dic[head_sub] = body_sub
                    data.append(tsv_dic)
                    line = f.readline()
            return data
 
 
if __name__ == "__main__":
    with codecs.open("tsv", 'w', "utf-8") as f:
        rows = """
        #Id\tContent
        #1\tContent1
        #2\tContent2
        #3\tContent3
        #4\tContent4
        #1024\tContent1024
        #"""
        f.writelines(rows.replace(' ', ''))
    TSV_Tester = TSV(file="ExpedientesMedicosCoNLL_Format.tsv") # Se importa el archivo de notas médicas con formato conll 
    #print(TSV_Tester.tsv())
   

In [ ]:
# Se carga el archivo y se separa en palabra y etiqueta
# Obtenemos posteriormente por error del código agregamos
# el contenido perdido

Exp_Med = TSV_Tester.tsv()

data = pd.DataFrame(Exp_Med)
data1 = np.array(Exp_Med)

lista_Palabra = list(data['"Nota'])
lista_Etiqueta = list(data['O'])

ls_tupla = []
for i in range(len(lista_Palabra)):
    ls_tupla.append([lista_Palabra[i], lista_Etiqueta[i]])
ls_tupla.insert(0, ['"Nota', 'O'])

In [ ]:
# Se realiza una separación para oraciones, se toma en cuenta cuando termina en un punto para
# hacer la separación

list_Notas_1 = []
ls_1 = []
for i in ls_tupla:
   
    if '"Nota' not in i:
        ls_1.append(i)
    if '"Nota' in i:
        
        ls_1.insert(0, ['"Nota', 'O'])
        list_Notas_1.append(ls_1)
        ls_1.append(['.', 'O'])
        ls_1=[]
list_Notas_1.pop([0][0])

[['"Nota', 'O'], ['.', 'O']]

In [ ]:
list_Notas_2 = []
list_Notas_2 = list_Notas_1
list_Notas_2[-10:] # Revisamos como se ve una nota

[[['"Nota', 'O'],
  ['Médica', 'O'],
  ['<', 'O'],
  ['paciente>', 'O'],
  ['7000000000487017', 'O'],
  ['22/03/1943', 'O'],
  ['518731', 'O'],
  ['<', 'O'],
  ['paciente>', 'O'],
  ['Mujer', 'O'],
  ['<', 'O'],
  ['doctor>', 'O'],
  ['HOSPITAL', 'O'],
  ['ABC', 'O'],
  ['<', 'O'],
  ['doctor>', 'O'],
  ['Signos', 'O'],
  ['Vitales', 'O'],
  ['26/07/2020', 'O'],
  ['01:53:', 'O'],
  ['26/07/2020', 'O'],
  ['06:10:', 'O'],
  ['Temperatura:', 'O'],
  ['36.7', 'Temperatura'],
  ['/', 'O'],
  ['Frecuencia', 'O'],
  ['cardiaca', 'O'],
  ['-', 'O'],
  ['ADL:', 'O'],
  ['78.0', 'FC'],
  ['/', 'O'],
  ['Frecuencia', 'O'],
  ['respiratoria', 'O'],
  ['-', 'O'],
  ['ADL:', 'O'],
  ['19.0', 'FR'],
  ['/', 'O'],
  ['SaO2:', 'O'],
  ['94.0', 'SATO2'],
  ['/', 'O'],
  ['Otras', 'O'],
  ['constantes', 'O'],
  ['de', 'O'],
  ['hoy:Tensión', 'O'],
  ['Arterial', 'O'],
  ['Sistólica', 'O'],
  ['-', 'O'],
  ['ADL:', 'O'],
  ['131.0', 'TAS'],
  ['/', 'O'],
  ['Tensión', 'O'],
  ['Arterial', 'O'],
  ['

In [ ]:
# Deshechamos las etiquetas y nos quedamos sólo con las de
# Síntomas, para sólo contemplar el parámetro que deseamos extraer
for i in list_Notas_2:
    for j in i:
        if "Sintomas" not in j:
            j[1] = 'O'

In [ ]:
# Realizamos una modificación para tener un formato BIO
for i in range(len(list_Notas_2)):
    c = 0
    #d = 0

    for j in range(len(list_Notas_2[i])):
        #print(list_Notas_2[i][j][1])


        if list_Notas_2[i][j][1] == 'Sintomas' and list_Notas_2[i][j-1][1] == 'O':
            #print('Hola')
            list_Notas_2[i][j][1] = 'B-SIN'
            c =+1

        if list_Notas_2[i][j][1] == 'Sintomas' and list_Notas_2[i][j-1][1] == 'B-SIN':
            #print('yy')
            list_Notas_2[i][j][1] = 'I-SIN'

        if list_Notas_2[i][j][1] == 'Sintomas' and list_Notas_2[i][j-1][1] == 'I-SIN':
            #print('yy')
            list_Notas_2[i][j][1] = 'I-SIN'

                

In [ ]:
list_Notas_2[-15:]

[[['"Nota', 'O'],
  ['Médica', 'O'],
  ['<', 'O'],
  ['paciente>', 'O'],
  ['7000000000486942', 'O'],
  ['25/01/1947', 'O'],
  ['517471', 'O'],
  ['<', 'O'],
  ['paciente>', 'O'],
  ['Mujer', 'O'],
  ['<', 'O'],
  ['doctor>', 'O'],
  ['HOSPITAL', 'O'],
  ['ABC', 'O'],
  ['<', 'O'],
  ['doctor>', 'O'],
  ['Signos', 'O'],
  ['Vitales', 'O'],
  ['26/07/2020', 'O'],
  ['06:24:', 'O'],
  ['Temperatura:', 'O'],
  ['36.2', 'O'],
  ['/', 'O'],
  ['Frecuencia', 'O'],
  ['cardiaca', 'O'],
  ['-', 'O'],
  ['ADL:', 'O'],
  ['60', 'O'],
  ['.0', 'O'],
  ['/', 'O'],
  ['Frecuencia', 'O'],
  ['respiratoria', 'O'],
  ['-', 'O'],
  ['ADL:', 'O'],
  ['19', 'O'],
  ['.0', 'O'],
  ['/', 'O'],
  ['SaO2:', 'O'],
  ['94', 'O'],
  ['.0', 'O'],
  ['/', 'O'],
  ['21/07/2020', 'O'],
  ['12:41:', 'O'],
  ['Peso', 'O'],
  ['(kg):', 'O'],
  ['70', 'O'],
  ['.0', 'O'],
  ['/', 'O'],
  ['Talla', 'O'],
  ['(cm):', 'O'],
  ['154', 'O'],
  ['.0', 'O'],
  ['/', 'O'],
  ['Otras', 'O'],
  ['constantes', 'O'],
  ['de', 'O'

In [ ]:
print('El número de notas es: ', len(list_Notas_2), 'notas médicas')

El número de notas es:  97 notas médicas


In [ ]:
# Elegimos el tamaño de nuestros conjuntos de prueva y entrenamiento.

Lis_Et = list_Notas_2

test_ratio = 0.1

train_ratio = int((1.0-test_ratio)*len(Lis_Et))



Lis_Et_train = Lis_Et[0:train_ratio]
Lis_Et_test = Lis_Et[train_ratio:]

In [ ]:
%%time
# Se convierte en listas 
train_sents = list(Lis_Et_train)
test_sents = list(Lis_Et_test)

Wall time: 0 ns


In [ ]:
import csv
import pandas as pd

In [ ]:
# Importamos el archivo con las etiquetas POS


df_Et = pd.read_csv('NotasS_E_P_T_.csv')
print(df_Et['POS'])

0        PUNCT
1        PROPN
2        PUNCT
3         NOUN
4          NUM
         ...  
47470     NOUN
47471    PROPN
47472     NOUN
47473      ADJ
47474    PUNCT
Name: POS, Length: 47475, dtype: object


In [ ]:
Lista_Etiqueta_POS =[]
Lista_Etiqueta_POS = list(df_Et['POS'])
Lista_Etiqueta_POS

['PUNCT',
 'PROPN',
 'PUNCT',
 'NOUN',
 'NUM',
 'NOUN',
 'NUM',
 'PUNCT',
 'NOUN',
 'NOUN',
 'PUNCT',
 'NOUN',
 'PROPN',
 'PROPN',
 'PUNCT',
 'NOUN',
 'NOUN',
 'ADJ',
 'NUM',
 'NUM',
 'NOUN',
 'NUM',
 'PUNCT',
 'NOUN',
 'ADJ',
 'PUNCT',
 'PROPN',
 'NUM',
 'PUNCT',
 'NOUN',
 'ADJ',
 'PUNCT',
 'PROPN',
 'NUM',
 'PUNCT',
 'NOUN',
 'NUM',
 'PUNCT',
 'PRON',
 'ADJ',
 'ADP',
 'ADV',
 'PROPN',
 'PROPN',
 'PUNCT',
 'PROPN',
 'NUM',
 'PUNCT',
 'PROPN',
 'PROPN',
 'PROPN',
 'PUNCT',
 'PROPN',
 'NUM',
 'PUNCT',
 'PROPN',
 'PROPN',
 'ADJ',
 'PUNCT',
 'PROPN',
 'NUM',
 'PUNCT',
 'PROPN',
 'PRON',
 'VERB',
 'ADP',
 'PUNCT',
 'NOUN',
 'ADP',
 'NUM',
 'PROPN',
 'ADP',
 'NOUN',
 'NOUN',
 'NOUN',
 'NOUN',
 'CCONJ',
 'PUNCT',
 'ADP',
 'PRON',
 'NOUN',
 'ADP',
 'NOUN',
 'NOUN',
 'DET',
 'NOUN',
 'ADJ',
 'VERB',
 'NOUN',
 'PROPN',
 'PROPN',
 'PROPN',
 'ADJ',
 'ADJ',
 'DET',
 'NOUN',
 'VERB',
 'DET',
 'NOUN',
 'ADP',
 'ADJ',
 'NOUN',
 'NOUN',
 'ADP',
 'NOUN',
 'ADJ',
 'ADJ',
 'DET',
 'PROPN',
 'ADJ',
 'NOUN

In [ ]:
len(list_Notas_1)
list_Notas_2 = []
list_Notas_2 = list_Notas_1
etiq_POS = []
etiq_POS

[]

In [ ]:
etiq_POS = list_Notas_2
etiq_POS[4]

[['"Nota', 'O'],
 ['Médica', 'O'],
 ['<', 'O'],
 ['paciente>', 'O'],
 ['7000000000486932', 'O'],
 ['22/06/1992', 'O'],
 ['517271', 'O'],
 ['<', 'O'],
 ['paciente>', 'O'],
 ['Mujer', 'O'],
 ['<', 'O'],
 ['doctor>', 'O'],
 ['HOSPITAL', 'O'],
 ['ABC', 'O'],
 ['<', 'O'],
 ['doctor>', 'O'],
 ['Signos', 'O'],
 ['Vitales', 'O'],
 ['22/07/2020', 'O'],
 ['02:26:', 'O'],
 ['22/07/2020', 'O'],
 ['08:37:', 'O'],
 ['Temperatura:', 'O'],
 ['36.7', 'O'],
 ['/', 'O'],
 ['Frecuencia', 'O'],
 ['cardiaca', 'O'],
 ['-', 'O'],
 ['ADL:', 'O'],
 ['75.0', 'O'],
 ['/', 'O'],
 ['Frecuencia', 'O'],
 ['respiratoria', 'O'],
 ['-', 'O'],
 ['ADL:', 'O'],
 ['16.0', 'O'],
 ['/', 'O'],
 ['SaO2:', 'O'],
 ['95.0', 'O'],
 ['/', 'O'],
 ['Otras', 'O'],
 ['constantes', 'O'],
 ['de', 'O'],
 ['hoy:Tensión', 'O'],
 ['Arterial', 'O'],
 ['Sistólica', 'O'],
 ['-', 'O'],
 ['ADL:', 'O'],
 ['103.0', 'O'],
 ['/', 'O'],
 ['Tensión', 'O'],
 ['Arterial', 'O'],
 ['Diastólica', 'O'],
 ['-', 'O'],
 ['ADL:', 'O'],
 ['64.0', 'O'],
 ['/',

In [ ]:
# Utilizamos el archivo creado con esta etiqueta y la añadimos 

contador_etiqueta = 0
for i in etiq_POS:
    for j in i:
        j.insert(1, Lista_Etiqueta_POS[contador_etiqueta])
        contador_etiqueta += 1

#Visualizamos el conjunto con la nota POS

etiq_POS[1]

[['"Nota', 'PUNCT', 'O'],
 ['Médica', 'PROPN', 'O'],
 ['<', 'PUNCT', 'O'],
 ['paciente>', 'NOUN', 'O'],
 ['7000000000486898', 'NUM', 'O'],
 ['07/05/1967', 'NUM', 'O'],
 ['516592', 'NUM', 'O'],
 ['<', 'PUNCT', 'O'],
 ['paciente>', 'NOUN', 'O'],
 ['Mujer', 'NOUN', 'O'],
 ['<', 'PUNCT', 'O'],
 ['doctor>', 'NOUN', 'O'],
 ['HOSPITAL', 'PROPN', 'O'],
 ['ABC', 'PROPN', 'O'],
 ['<', 'PUNCT', 'O'],
 ['doctor>', 'NOUN', 'O'],
 ['Signos', 'NOUN', 'O'],
 ['Vitales', 'ADJ', 'O'],
 ['21/07/2020', 'NUM', 'O'],
 ['06:50:', 'NUM', 'O'],
 ['Temperatura:', 'NOUN', 'O'],
 ['36.3', 'NUM', 'O'],
 ['/', 'PUNCT', 'O'],
 ['Frecuencia', 'NOUN', 'O'],
 ['cardiaca', 'ADJ', 'O'],
 ['-', 'PUNCT', 'O'],
 ['ADL:', 'PROPN', 'O'],
 ['52.0', 'NUM', 'O'],
 ['/', 'PUNCT', 'O'],
 ['Frecuencia', 'NOUN', 'O'],
 ['respiratoria', 'ADJ', 'O'],
 ['-', 'PUNCT', 'O'],
 ['ADL:', 'PROPN', 'O'],
 ['22.0', 'NUM', 'O'],
 ['/', 'PUNCT', 'O'],
 ['SaO2:', 'NOUN', 'O'],
 ['95.0', 'NUM', 'O'],
 ['/', 'PUNCT', 'O'],
 ['16/07/2020', 'NUM', '

In [ ]:
# Separamos cada una de las notas en Oraciones, consideramos el punto como el fin de una oración
list_ = etiq_POS
list_Notas_11 = []
ls_11 = []
ls_2 = []
for i in list_:
    for j in i:
        if '.' not in j[0]:
            ls_11.append(j)
        if '.' in j[0]:
            #ls_1.insert(0, ['"Nota', 'O'])
            ls_11.append(j)
            list_Notas_11.append(ls_11)
            ls_11=[]
    ls_2.append(list_Notas_11)
    list_Notas_11 = []

In [ ]:
# Visualizamos 
ls_2[1][1]

[['/', 'PUNCT', 'O'],
 ['Frecuencia', 'NOUN', 'O'],
 ['cardiaca', 'ADJ', 'O'],
 ['-', 'PUNCT', 'O'],
 ['ADL:', 'PROPN', 'O'],
 ['52.0', 'NUM', 'O']]

In [ ]:
# Extraemos sólo las oraciones que tienen síntomas

Lista_Ora_Sin = []

for i in ls_2:
  for j in i:
    c = 0
    for k in j: 
      if 'O' not in k:
        if c == 0:
          Lista_Ora_Sin.append(j)
          c = c + 1

print('El número de oraciones que tienen síntomas son: ', len(Lista_Ora_Sin), 'oraciones')

El número de oraciones que tienen síntomas son:  79 oraciones


In [ ]:
# El preprocesamiento para introducir en CRF
def word2features_(sent, i):
    word = sent[i][0]
    
    
    
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features_(sent):
    return [word2features_(sent, i) for i in range(len(sent))]

def sent2labels_(sent):
    return [label for token, postag, label in sent]

def sent2tokens_(sent):
    return [token for token, postag, label in sent]

In [ ]:
# Separamos en conjuntos de Prueba y entrenamiento

Lis_Et_1 = Lista_Ora_Sin

test_ratio_1 = 0.1

train_ratio_1 = int((1.0-test_ratio_1)*len(Lis_Et_1))



Lis_Et_train_1 = Lis_Et_1[0:train_ratio_1]
Lis_Et_test_1 = Lis_Et_1[train_ratio_1:]

In [ ]:
%%time
train_sents_1 = list(Lis_Et_train_1)
test_sents_1 = list(Lis_Et_test_1)

Wall time: 1 ms


In [ ]:
# Revisamos una oración del conjunto de entrenamiento
train_sents_1[0]

[['Al', 'ADP', 'O'],
 ['pase', 'NOUN', 'O'],
 ['de', 'ADP', 'O'],
 ['visita', 'NOUN', 'O'],
 ['se', 'PRON', 'O'],
 ['refiere', 'VERB', 'O'],
 ['asintomática', 'PROPN', 'B-SIN'],
 ['Objetivo', 'NOUN', 'O'],
 ['Femenino', 'ADJ', 'O'],
 ['de', 'ADP', 'O'],
 ['edad', 'NOUN', 'O'],
 ['aparente', 'ADJ', 'O'],
 ['similar', 'ADJ', 'O'],
 ['la', 'DET', 'O'],
 ['cronológica,', 'PROPN', 'O'],
 ['consciente,', 'ADJ', 'O'],
 ['orientada.', 'ADJ', 'O']]

In [ ]:
test_sents_1

[[['Al', 'ADP', 'O'],
  ['pase', 'NOUN', 'O'],
  ['de', 'ADP', 'O'],
  ['visita', 'NOUN', 'O'],
  ['la', 'DET', 'O'],
  ['paciente', 'NOUN', 'O'],
  ['refiere', 'VERB', 'O'],
  ['mejoria,', 'NOUN', 'O'],
  ['Niega', 'VERB', 'B-SIN'],
  ['cefalea,', 'NOUN', 'O'],
  ['tos,', 'NOUN', 'O'],
  ['dolor', 'NOUN', 'O'],
  ['torácico,', 'PROPN', 'O'],
  ['sintomatología', 'PROPN', 'O'],
  ['urinaria', 'ADJ', 'O'],
  ['gastrointestinal', 'ADJ', 'O'],
  ['Objetivo', 'NOUN', 'O'],
  ['Mujer', 'NOUN', 'O'],
  ['de', 'ADP', 'O'],
  ['edad', 'NOUN', 'O'],
  ['aparente', 'ADJ', 'O'],
  ['igual', 'ADJ', 'O'],
  ['la', 'DET', 'O'],
  ['cronológica,', 'PROPN', 'O'],
  ['orientada', 'ADJ', 'O'],
  ['en', 'NOUN', 'O'],
  ['tiempo,', 'NOUN', 'O'],
  ['persona,', 'NOUN', 'O'],
  ['lugar', 'NOUN', 'O'],
  ['circunstancia,', 'NOUN', 'O'],
  ['alerta.', 'NOUN', 'O']],
 [['Afebril', 'PROPN', 'B-SIN'], ['.', 'PUNCT', 'O']],
 [['Negó', 'PROPN', 'B-SIN'],
  ['sintomatología', 'PROPN', 'I-SIN'],
  ['urinaria', 

In [ ]:
# Aplicamos el preprocesamiento a cada una de las palabras
#%%time
X_train_1 = [sent2features_(s) for s in train_sents_1]
y_train_1 = [sent2labels_(s) for s in train_sents_1]

X_test_1 = [sent2features_(s) for s in test_sents_1]
y_test_1 = [sent2labels_(s) for s in test_sents_1]

In [ ]:
# Entrenamos el modelo de CRF
#%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    all_possible_transitions=True,
    c1=0.18182582453025384,
    c2=0.00818676688659457,
    #keep_tempfiles=None,
    max_iterations=100    
)
#try:
crf.fit(X_train_1, y_train_1)
#except AttributeError:
#    pass
predictions = crf.predict(X_test_1)

In [ ]:
# Revisamos las etiquetas que tenemos y removemos a O sólo para que no entre en nuestro cálculos
labels = list(crf.classes_)
labels.remove('O')
labels

['B-SIN', 'I-SIN']

In [ ]:
# Realizamos predicciones
y_pred_1 = crf.predict(X_test_1)
metrics.flat_f1_score(y_test_1, y_pred_1,
                      average='weighted', labels=labels)

0.7335858585858587

In [ ]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test_1, y_pred_1, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

       B-SIN      0.889     0.615     0.727        13
       I-SIN      1.000     0.600     0.750         5

   micro avg      0.917     0.611     0.733        18
   macro avg      0.944     0.608     0.739        18
weighted avg      0.920     0.611     0.734        18



C:\Users\avata\anaconda3\envs\python3.7\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['B-SIN', 'I-SIN'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [ ]:
# Procesamos para ver las predicciones que está haciendo 

Palabra = []
for i in test_sents_1:
    for j in i:
        Palabra.append(j[0])

Test = []
for i in y_test_1:
    for j in i:
        Test.append(j)

Predicciones = []
for i in y_pred_1:
    for j in i:
        Predicciones.append(j)

data_frame = pd.DataFrame(list(zip(Palabra, Test, Predicciones)), columns = ['Palabra','Test','Predicción'])

In [ ]:
li_1 = list(zip(Palabra, Test, Predicciones))

In [ ]:
Lista_oraciones_prediccionesn1 = []
contador1 = 0
for i in li_1:
  if 'B-SIN' == i[1] or 'I-SIN' == i[1]:
    Lista_oraciones_prediccionesn1.append(i)
  #contador1 += 1
  

In [ ]:
Tabla_predicciones_CRF = pd.DataFrame( Lista_oraciones_prediccionesn1, columns = ['Palabra','Test','Predicción'])
Tabla_predicciones_CRF

,Palabra,Test,Predicción
0,Niega,B-SIN,O
1,Afebril,B-SIN,B-SIN
2,Negó,B-SIN,B-SIN
3,sintomatología,I-SIN,I-SIN
4,insuficiencia,B-SIN,O
5,respiratoria,I-SIN,O
6,odinofagia,B-SIN,O
7,tos,B-SIN,B-SIN
8,seca,I-SIN,I-SIN
9,cefalea,B-SIN,B-SIN


# Tratamiento de notas json.

In [ ]:
with open('HG_Cabrera_samih_covid_10_08_2020.json') as file:
    data = json.load(file)
data['1']['nota']

{'Signos Vitales': '\n\n\n10/09/2020 06:49: Temperatura: 36.4      /      Frecuencia cardi\naca - ADL: 64.0      /      Frecuencia respiratoria - ADL: 19.0      /      SaO2\n: 93.0      /                                                                   \nOtras constantes de hoy:Tensión Arterial Sistólica - ADL: 100.0      /      Te\nnsión Arterial Diastólica - ADL: 60.0      /      Tensión Arterial Media - AD\nL: 73.0      /      Glicemia capilar: 115.0      /\n',
 'Síntomas': '\n\n\nSe trata de Arellano Ojeda Marilu, paciente femenino de 52 años\n de edad, con antecedente de Diabetes Mellitus tipo 2 de 2 años de diagnóstico\n en tratamiento regular con Vidagliptina y Metformina, sobrepeso.               \nAl pase de visita la paciente niega síntomas respiratorios como disnea o tos, n\niega presencia de dolor o fiebre,, uresis y evacuaciones al corriente.\n',
 'Objetivo': '\n\n\nFemenina de edad aparente igual a la cronológica, alerta, orien\ntada y cooperadora, con adecuada coloració

In [ ]:
with open('HG_Cabrera_samih_covid_10_08_2020.json') as file:
    data = json.load(file)
Lista_notas_Cabrera = []
ls = []
for i in data:
    #for j in data[i]['nota']['Síntomas']:
    ls.append(('Síntomas:'+ data[i]['nota']['Síntomas']).split())
        #ls.append((str(j) +':' + data[i]['nota'][j]).split())
for i in ls:
    l_v = []
    for j in i:
        l_v.append([j])
    Lista_notas_Cabrera.append(l_v)
    #Lista_notas_Cabrera.append(ls)

# El preprocesamiento para introducir en CRF
def word2features_(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
    #    'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
    #    postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
    #        '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
        #    '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features_(sent):
    return [word2features_(sent, i) for i in range(len(sent))]



In [ ]:
# El preprocesamiento para introducir en CRF
def word2features_(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
    #    'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
    #        '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
        #    '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features_(sent):
    return [word2features_(sent, i) for i in range(len(sent))]


In [ ]:
nlp = spacy.load("es_dep_news_trf")

In [ ]:
len(Lista_notas_Cabrera)

292

In [ ]:
lis_etiquetaPOS = []
contador = 0
for i in Lista_notas_Cabrera:

    for j in i:
        
        text = 0
        text = nlp(j[0])
        al = 0
        al = [(w.pos_) for w in text]
        al[0]
        lis_etiquetaPOS.append(al[0]) #j.insert(1, al[0])
        #j.insert(0, b)
        #lis_etiquetaPOS.append(a)
    contador += 1
    print(contador)
    #print(text_[0][1])

C:\Users\avata\anaconda3\envs\python3.7\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
# Utilizamos el archivo creado con esta etiqueta

contador_etiqueta = 0
for i in Lista_notas_Cabrera:
    for j in i:
        j.insert(1, lis_etiquetaPOS[contador_etiqueta])
        contador_etiqueta += 1

In [ ]:
Lista_notas_Cabrera[-5]

[['Síntomas:', 'NOUN'],
 ['Se', 'PRON'],
 ['trata', 'VERB'],
 ['de', 'ADP'],
 ['Arturo', 'PROPN'],
 ['de', 'ADP'],
 ['53', 'NUM'],
 ['años', 'NOUN'],
 ['de', 'ADP'],
 ['edad,', 'NOUN'],
 ['niega', 'VERB'],
 ['antecedentes', 'NOUN'],
 ['de', 'ADP'],
 ['en', 'NOUN'],
 ['fermedades', 'NOUN'],
 ['crónico', 'ADJ'],
 ['degenerativas.', 'ADJ'],
 ['Al', 'ADP'],
 ['pase', 'NOUN'],
 ['de', 'ADP'],
 ['visita', 'NOUN'],
 ['el', 'DET'],
 ['paciente', 'NOUN'],
 ['niega', 'VERB'],
 ['dolor', 'NOUN'],
 ['torácico.', 'ADJ'],
 ['Negó', 'VERB'],
 ['sintomatología', 'NOUN'],
 ['urina', 'PROPN'],
 ['ria', 'PROPN'],
 ['o', 'CCONJ'],
 ['digestiva.', 'ADJ'],
 ['El', 'DET'],
 ['paciente', 'NOUN'],
 ['niega', 'VERB'],
 ['la', 'DET'],
 ['presencia', 'NOUN'],
 ['de', 'ADP'],
 ['disnea.', 'ADJ']]

In [ ]:
len(Lista_notas_Cabrera)

292

In [ ]:
data_f_POS_Nota = pd.DataFrame(Lista_notas_Cabrera, index=None)

In [ ]:
data_f_POS_Nota

,0,1,2,3,4,5,6,7,8,9,...,306,307,308,309,310,311,312,313,314,315
0,"[Síntomas:, NOUN]","[Se, PRON]","[trata, VERB]","[de, ADP]","[Arellano, PROPN]","[Ojeda, PROPN]","[Marilu,, PROPN]","[paciente, NOUN]","[femenino, ADJ]","[de, ADP]",...,None,None,None,None,None,None,None,None,None,None
1,"[Síntomas:, NOUN]","[Se, PRON]","[trata, VERB]","[de, ADP]","[Leticia, PROPN]","[Hernández, PROPN]","[Rosas, PROPN]","[de, ADP]","[64, NUM]","[años, NOUN]",...,None,None,None,None,None,None,None,None,None,None
2,"[Síntomas:, NOUN]","[Se, PRON]","[trata, VERB]","[de, ADP]","[Arellano, PROPN]","[Ojeda,, PROPN]","[Marilu, PROPN]","[de, ADP]","[52, NUM]","[años, NOUN]",...,None,None,None,None,None,None,None,None,None,None
3,"[Síntomas:, NOUN]","[Se, PRON]","[trata, VERB]","[de, ADP]","[Rosa,, PROPN]","[femenino, ADJ]","[de, ADP]","[46, NUM]","[años, NOUN]","[de, ADP]",...,None,None,None,None,None,None,None,None,None,None
4,"[Síntomas:, NOUN]","[Se, PRON]","[trata, VERB]","[de, ADP]","[Mónica, PROPN]","[Alejandra, PROPN]","[Cano, PROPN]","[Tierrasnegras, PROPN]","[de, ADP]","[55, NUM]",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,"[Síntomas:, NOUN]","[Se, PRON]","[trata, VERB]","[de, ADP]","[Arturo, PROPN]","[de, ADP]","[53, NUM]","[años, NOUN]","[de, ADP]","[edad,, NOUN]",...,None,None,None,None,None,None,None,None,None,None
288,"[Síntomas:, NOUN]","[Se, PRON]","[trata, VERB]","[de, ADP]","[Esther, PROPN]","[Flores, NOUN]","[Perez, PROPN]","[de, ADP]","[25, NUM]","[años, NOUN]",...,None,None,None,None,None,None,None,None,None,None
289,"[Síntomas:, NOUN]","[Se, PRON]","[trata, VERB]","[de, ADP]","[Maribel, PROPN]","[Martínez, PROPN]","[Ramírez, PROPN]","[de, ADP]","[33, NUM]","[años, NOUN]",...,None,None,None,None,None,None,None,None,None,None
290,"[Síntomas:, NOUN]","[Se, PRON]","[trata, VERB]","[de, ADP]","[Maribel, PROPN]","[Martínez, PROPN]","[Ramírez, PROPN]","[de, ADP]","[33, NUM]","[años, NOUN]",...,None,None,None,None,None,None,None,None,None,None


In [ ]:
data_f_POS_Nota.to_csv('POS_Cabrera_Sintomas.csv')

In [ ]:
ls_1 = []
[[[ls_1.append(k) for k in i] for i in j] for j in Lista_notas_Cabrera]
#print(ls_1)

[]

In [ ]:
Lista_Cabrera = [sent2features_(s) for s in Lista_notas_Cabrera]
Lista_Cabrera[1]

[{'bias': 1.0,
  'word.lower()': 'síntomas:',
  'word[-3:]': 'as:',
  'word[-2:]': 's:',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'NOUN',
  'BOS': True,
  '+1:word.lower()': 'se',
  '+1:word.istitle()': True,
  '+1:word.isupper()': False,
  '+1:postag': 'PRON'},
 {'bias': 1.0,
  'word.lower()': 'se',
  'word[-3:]': 'Se',
  'word[-2:]': 'Se',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'PRON',
  '-1:word.lower()': 'síntomas:',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '-1:postag': 'NOUN',
  '+1:word.lower()': 'trata',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'VERB'},
 {'bias': 1.0,
  'word.lower()': 'trata',
  'word[-3:]': 'ata',
  'word[-2:]': 'ta',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'VERB',
  '-1:word.lower()': 'se',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '

In [ ]:
Predicciones_notas_cabrera = crf.predict(Lista_Cabrera)
Predicciones_notas_cabrera

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-SIN',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',


In [ ]:
Palabra = []
for i in Lista_notas_Cabrera:
    for j in i:
        Palabra.append(j[0])


Predicciones = []
for i in Predicciones_notas_cabrera:
    for j in i:
        Predicciones.append(j)

li_1 = []
li_1 = list(zip(Palabra,  Predicciones))
Lista_oraciones_prediccionesn1 = []
contador1 = 0
for i in li_1:
  if 'B-SIN' == i[1] or 'I-SIN' == i[1]:
    Lista_oraciones_prediccionesn1.append(i)
  #contador1 += 1SS
Tabla_predicciones_CRF = pd.DataFrame( Lista_oraciones_prediccionesn1, columns = ['Palabra','Predicción'])
Tabla_predicciones_CRF

,Palabra,Predicción
0,disnea,B-SIN
1,disnea,B-SIN
2,niega,B-SIN
3,dolor,I-SIN
4,torácico,I-SIN
...,...,...
89,disnea,B-SIN
90,tos,B-SIN
91,seca,I-SIN
92,cefalea,B-SIN


In [ ]:
Tabla_predicciones_CRF[:45]

,Palabra,Predicción
0,disnea,B-SIN
1,disnea,B-SIN
2,niega,B-SIN
3,dolor,I-SIN
4,torácico,I-SIN
5,disnea,B-SIN
6,disnea,B-SIN
7,disnea,B-SIN
8,tos,B-SIN
9,disnea,B-SIN


In [ ]:
Tabla_predicciones_CRF[45:]

,Palabra,Predicción
45,disnea,B-SIN
46,cefalea,B-SIN
47,disnea,B-SIN
48,disnea,B-SIN
49,niega,B-SIN
50,dolor,I-SIN
51,torácico,I-SIN
52,tos,B-SIN
53,tos,B-SIN
54,tos,B-SIN
